# Multi-Agent Customer Service System
 
## Overview
This notebook demonstrates how to build a **multi-agent system** using Semantic Kernel where different specialized agents handle specific types of customer inquiries.

<div align="center">
<img src="lesson_1.png" alt="Alt text" width="800"/>
</div>
 
### Key Concepts Covered:
1. **Agent Specialization**: Each agent has a specific domain of expertise
2. **Boundary Enforcement**: Agents decline requests outside their domain
3. **Service Configuration**: Setting up Azure OpenAI with custom parameters
4. **Asynchronous Processing**: Using async/await for efficient agent communication

## 1. Setup and Configuration

In [1]:
import os
from dotenv import load_dotenv

from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, OpenAIChatPromptExecutionSettings
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.functions import KernelArguments

# Load environment variables from .env file
load_dotenv()

# Azure OpenAI Configuration
AZURE_OPENAI_KEY = os.getenv("AZURE_OPENAI_KEY")
BASE_URL = os.getenv("URL")
API_VERSION = "2024-12-01-preview"
DEPLOYMENT = "gpt-4.1-mini"

## 2. Initialize Kernel and AI Service
 
The **Kernel** is the core orchestration layer that manages services and plugins.
#We configure an Azure OpenAI chat service and register it with the kernel.

In [2]:
kernel = Kernel()

# Create Azure OpenAI chat service
chat_service = AzureChatCompletion(
    deployment_name=DEPLOYMENT,
    api_key=AZURE_OPENAI_KEY,
    base_url=f"{BASE_URL}{DEPLOYMENT}",
    api_version=API_VERSION
)

# Register the service with the kernel
kernel.add_service(chat_service)

## 3. Agent Factory Function
 
This helper function creates agents with customized settings:
- **temperature**: Controls response randomness (0.0 = deterministic, 1.0 = creative)
- **max_tokens**: Limits response length

In [3]:
def create_agent(name: str, instructions: str, temperature=0.7, max_tokens=400):
    """
    Creates a ChatCompletionAgent with specified parameters.
    
    Args:
        name: Agent identifier
        instructions: System prompt defining agent's role and boundaries
        temperature: Response creativity (default: 0.7)
        max_tokens: Maximum response length (default: 400)
    
    Returns:
        ChatCompletionAgent instance
    """
    settings = OpenAIChatPromptExecutionSettings()
    settings.temperature = temperature
    settings.max_tokens = max_tokens
    
    return ChatCompletionAgent(
        service=chat_service,
        name=name,
        instructions=instructions,
        arguments=KernelArguments(settings)
    )

## 4. Define Specialized Agents
 
 Each agent is an expert in a specific customer service domain.
 Notice how each instruction explicitly defines:
 - What the agent **can** handle
 - What to do when encountering requests **outside** their expertise

### 4.1 Billing Agent

In [4]:
billing_agent = create_agent(
    "BillingAgent",
    """
You are a billing and payment specialist for a retail company.
If the question is about invoices, payments, refunds, charges, or billing issues, provide helpful answers.
If the question is NOT related to billing or payments, respond with: "I cannot help with this. Please contact the appropriate department."
""",
    temperature=0.5  # Lower temperature for consistent financial information
)

### 4.2 Technical Support Agent

In [5]:
tech_support_agent = create_agent(
    "TechSupportAgent",
    """
You are a technical support specialist.
If the question is about product troubleshooting, technical issues, setup, or how to use features, provide assistance.
If the question is NOT technical in nature, respond with: "I cannot help with this. Please contact the appropriate department."
""",
    temperature=0.6  # Slightly creative for problem-solving
)


### 4.3 Product Information Agent

In [6]:
product_info_agent = create_agent(
    "ProductInfoAgent",
    """
You are a product information specialist.
If the question is about product specifications, availability, features, or comparisons, answer it.
If the question is NOT about product information, respond with: "I cannot help with this. Please contact the appropriate department."
""",
    temperature=0.7  # Balanced for informative responses
)

### 4.4 Returns and Exchanges Agent

In [7]:
returns_agent = create_agent(
    "ReturnsAgent",
    """
You are a returns and exchanges specialist.
If the question is about return policies, exchange procedures, shipping returns, or product complaints, help the customer.
If the question is NOT about returns or exchanges, respond with: "I cannot help with this. Please contact the appropriate department."
""",
    temperature=0.5  # Consistent policy information
)

## 5. Collect all agents in a list for iteration

In [8]:
agents = [billing_agent, tech_support_agent, product_info_agent, returns_agent]

## 6. Run Customer Inquiries
 
We'll test various customer questions to see how each agent responds.
**Expected behavior**: Only the relevant agent provides a full answer; others decline.

In [9]:
async def main():
    """
    Simulates customer inquiries being routed to all agents.
    In production, you'd implement smart routing to send queries only to relevant agents.
    """
    
    customer_inquiries = [
        "Why was I charged twice on my last order?",
        "My laptop won't turn on after the latest update. What should I do?",
        "Do you have the wireless headphones in blue color?",
        "I want to return a defective product I bought last week",
        "What are the specs of your newest smartphone model?",
        "How can I track my refund status?"
    ]

    print("=" * 60)
    print("CUSTOMER SERVICE MULTI-AGENT ROUTING SYSTEM")
    print("=" * 60)
    
    for i, inquiry in enumerate(customer_inquiries, start=1):
        print(f"\n{'='*60}")
        print(f"Customer Inquiry #{i}")
        print(f"{'='*60}")
        print(f"Question: {inquiry}\n")
        
        # Route inquiry to all agents (in production, use smart routing)
        for agent in agents:
            print(f"[{agent.name}]")
            print("-" * 40)
            
            # Invoke agent asynchronously and stream response
            async for message in agent.invoke(inquiry):
                print(f"{message}")
            print()

## 7. Run the Demo

In [10]:
await main()

CUSTOMER SERVICE MULTI-AGENT ROUTING SYSTEM

Customer Inquiry #1
Question: Why was I charged twice on my last order?

[BillingAgent]
----------------------------------------


ServiceResponseException: ("<class 'semantic_kernel.connectors.ai.open_ai.services.azure_chat_completion.AzureChatCompletion'> service failed to complete the prompt", NotFoundError("Error code: 404 - {'error': {'code': '404', 'message': 'Resource not found'}}"))

 ## Key Takeaways
 
 ### 1. **Agent Specialization**
 Each agent has a narrow, well-defined role. This mirrors real-world customer service departments.
 
 ### 2. **Boundary Enforcement**
 Agents explicitly decline out-of-scope requests, preventing hallucinations and maintaining reliability.
 
 ### 3. **Scalability**
 This pattern scales easily: add new agents (e.g., "ShippingAgent") by following the same template.
 
 ### 4. **Production Considerations**
 - **Smart Routing**: Use a classifier agent or intent detection to route to the right agent first
 - **Fallback Agent**: Add a general agent for questions no specialist can handle
 - **Logging**: Track which agents handle which queries for analytics
 - **Human Escalation**: Some queries should trigger human agent involvement
 
 ### 5. **Alternative Patterns**
 - **Sequential Processing**: Route through agents in order until one handles it
 - **Voting System**: Multiple agents respond; best answer is selected
 - **Hierarchical**: Supervisor agent delegates to specialist agents